In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from datasets import load_dataset
import cassio
from PyPDF2 import PdfReader

e:\LangChainProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

ASTRA_DB_APPLICATION_TOKEN = ''
ASTRA_DB_ID = ''
os.environ['OpenAI_API_KEY'] =  ''

In [4]:
pdfreader = PdfReader('./Btc.pdf')

In [5]:
from typing_extensions import Concatenate

raw_text =""
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [7]:
llm = OpenAI()
embedding = OpenAIEmbeddings()

e:\LangChainProject\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
e:\LangChainProject\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name='demo',
    session=None,
    keyspace=None
)

In [9]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [10]:
texts[:50]

['econstor\nMake Your Publications Visible.A Service of\nzbwLeibniz-Informationszentrum\nWirtschaft\nLeibniz Information Centre\nfor Economics\nGbadebo, Adedeji Daniel; Adekunle, Ahmed Oluwatobi; Adedokun, Wole;\nLukman, Adebayo-Oke Abdulrauf; Akande, Joseph\nArticle\nBTC price volatility: Fundamentals versus information\nCogent Business & Management\nProvided in Cooperation with:\nTaylor & Francis Group\nSuggested Citation: Gbadebo, Adedeji Daniel; Adekunle, Ahmed Oluwatobi; Adedokun, Wole;\nLukman, Adebayo-Oke Abdulrauf; Akande, Joseph (2021) : BTC price volatility: Fundamentals\nversus information, Cogent Business & Management, ISSN 2331-1975, Taylor & Francis,\nAbingdon, Vol. 8, Iss. 1, pp. 1-21,\nhttps://doi.org/10.1080/23311975.2021.1984624\nThis Version is available at:',
 'versus information, Cogent Business & Management, ISSN 2331-1975, Taylor & Francis,\nAbingdon, Vol. 8, Iss. 1, pp. 1-21,\nhttps://doi.org/10.1080/23311975.2021.1984624\nThis Version is available at:\nhttp://h

In [13]:
astra_vector_store.add_texts(texts[:50])
print('Inserted %i headlines.' % len(texts[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [14]:
first_question = True
while True:
    if first_question:
        query_text = input("\nWhat is your query? or type 'exit' to quit: ").strip()
    else:
        query_text = input("\nWhat is your next query? or type 'exit' to quit: ").strip()

    if query_text.lower() == "exit":
        break  

    first_question = False

    print("\nQuestion: \"%s\"" % query_text) 
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("Answer: \"%s\"\n" % answer)

    print('First Doucment by relevance')

    for doc, score in astra_vector_store.similarity_search_with_score(query_text,k=4):
        print("   [%0.4f] \"%s ... \"" % (score, doc.page_content[:84]))